In [1]:
cd /Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/P3HT

/Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/P3HT


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm


In [3]:
dataset = pd.read_csv(r'/Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/P3HT/P3HT_dataset.csv')
X = dataset.iloc[:,5:10]
# i put negative sign here because default BO is for minimization
X['Conductivity'] = -dataset.iloc[:,13].values
X_run = X
X_run



,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
0,45.00,0.00,55.00,0.00,0.00,-12.77
1,75.00,0.00,25.00,0.00,0.00,-13.19
2,30.00,0.00,70.00,0.00,0.00,-14.78
3,30.00,0.00,70.00,0.00,0.00,-16.34
4,45.00,0.00,55.00,0.00,0.00,-16.94
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,-697.40
229,44.30,54.68,0.04,0.29,0.78,-731.11
230,38.92,49.28,0.09,0.95,10.67,-738.10
231,44.30,54.68,0.04,0.29,0.78,-772.94


In [4]:
features = ['P3HT content (%)', 'D1 content (%)','D2 content (%)','D6 content (%)','D8 content (%)']
X_feature = X_run[['P3HT content (%)', 'D1 content (%)','D2 content (%)','D6 content (%)','D8 content (%)']].values

X_Conductivity = np.array(X_run['Conductivity'].values)

In [5]:
bounds = [{'name':'P3HT content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D1 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D2 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D6 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D8 content (%)','type':'continuous','domain':(0, 100)}
         ]

constraints_baseline = [{'name': 'constr_1', 'constraint': 'x[:,0] + x[:,1] + x[:,2] + x[:,3] + x[:,4]- 100'}
                       ]

jitter = 0.01

# aaa = GPyOpt.util.general.normalize(np.array([[i] for i in X_Conductivity]), normalization_type='maxmin')

BO_teacher = GPyOpt.methods.BayesianOptimization(f=None,  
                                                 domain = bounds,
                                                 constraints = constraints_baseline,
                                                 acquisition_type = 'EI',              
                                                 normalize_Y = True,
                                                 X = np.array(X_feature) ,
                                                 Y = np.array([[i] for i in X_Conductivity]),
                                                 evaluator_type = 'local_penalization',
                                                 batch_size = 1,
                                                 acquisition_jitter = jitter)

# we don't care yet, this line here is to just fit the GP
_ = BO_teacher.suggest_next_locations()


# denormalazing
input_mean = np.mean(X_Conductivity)
input_std = np.std(X_Conductivity)



In [20]:
GP_ground_Conductivity = BO_teacher.model.predict(np.array(X_feature))[0][:,0] * input_std + input_mean
X_GP_ground = X_run.copy()
X_GP_ground['Conductivity'] = GP_ground_Conductivity
X_GP_ground

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
0,45.00,0.00,55.00,0.00,0.00,-11.348070
1,75.00,0.00,25.00,0.00,0.00,-1.245150
2,30.00,0.00,70.00,0.00,0.00,-36.425320
3,30.00,0.00,70.00,0.00,0.00,-36.425320
4,45.00,0.00,55.00,0.00,0.00,-11.348070
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,-477.556756
229,44.30,54.68,0.04,0.29,0.78,-598.547011
230,38.92,49.28,0.09,0.95,10.67,-509.717209
231,44.30,54.68,0.04,0.29,0.78,-598.547011


In [21]:
X_GP_ground.sort_values(['Conductivity']).head(25)

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
232,40.00,60.00,0.00,0.00,0.00,-692.194336
27,40.00,60.00,0.00,0.00,0.00,-692.194336
26,40.00,60.00,0.00,0.00,0.00,-692.194336
25,40.00,60.00,0.00,0.00,0.00,-692.194336
23,40.00,60.00,0.00,0.00,0.00,-692.194336
21,40.00,60.00,0.00,0.00,0.00,-692.194336
20,40.00,60.00,0.00,0.00,0.00,-692.194336
19,40.00,60.00,0.00,0.00,0.00,-692.194336
16,40.00,60.00,0.00,0.00,0.00,-692.194336
226,40.67,59.26,0.04,0.04,0.07,-679.286819


In [6]:
import xgboost as xgb
# https://xgboost.readthedocs.io/en/latest/parameter.html
num_round = 3000
dtrain = xgb.DMatrix(np.array(X_feature), label=X_Conductivity)


param = {'max_depth': 6, 'eta': 0.3, 'objective': 'reg:squarederror'}
bst = xgb.train(param, dtrain, num_round)


XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/liangqiaohao/opt/anaconda3/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/liangqiaohao/opt/anaconda3/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [23]:
XGB_ground_Conductivity = bst.predict(xgb.DMatrix(np.array(X_feature)))
X_XGB_ground = X_run.copy()
X_XGB_ground['Conductivity'] = XGB_ground_Conductivity
X_XGB_ground

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
0,45.00,0.00,55.00,0.00,0.00,-14.855220
1,75.00,0.00,25.00,0.00,0.00,-13.191488
2,30.00,0.00,70.00,0.00,0.00,-15.559357
3,30.00,0.00,70.00,0.00,0.00,-15.559357
4,45.00,0.00,55.00,0.00,0.00,-14.855220
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,-697.400879
229,44.30,54.68,0.04,0.29,0.78,-649.948486
230,38.92,49.28,0.09,0.95,10.67,-687.780396
231,44.30,54.68,0.04,0.29,0.78,-649.948486


In [24]:
X_XGB_ground.sort_values(['Conductivity']).head(25)

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
111,46.92,50.30,1.53,0.04,1.23,-838.310242
110,40.29,58.82,0.29,0.41,0.19,-770.347534
109,47.81,49.29,0.55,1.46,0.89,-750.897034
194,43.61,49.29,2.88,4.22,0.04,-739.503235
193,43.61,49.29,2.88,4.22,0.04,-739.503235
27,40.00,60.00,0.00,0.00,0.00,-738.663147
26,40.00,60.00,0.00,0.00,0.00,-738.663147
25,40.00,60.00,0.00,0.00,0.00,-738.663147
23,40.00,60.00,0.00,0.00,0.00,-738.663147
21,40.00,60.00,0.00,0.00,0.00,-738.663147


In [25]:
# batch and rounds for BO
batch = 1
rounds = 100
n_eval = batch * rounds
jitter = 0.01

seed_list = [74,660,861,1526,2,151,19,7,33,4021,66,42424,671,25,95711,850,71,2148,26219,4]
# 20


In [26]:
import pyDOE
import random

X_BO_initRS_EI_collection = []
run_BO_initRS_EI_collection = []
Conductivity_BO_initRS_EI_collection = []

for s in seed_list:
    
    # initial_sampling = constrained random sampling = cRS
    # optimization = GpyOpt

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_EI = []
    run_BO_initRS_EI = []
    Conductivity_BO_initRS_EI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_EI.append(initial_X[j])
                run_BO_initRS_EI.append(i)
                Conductivity_BO_initRS_EI.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'EI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_EI),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_EI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_Conductivity = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            
            
            for j in np.arange(batch):
                X_BO_initRS_EI.append(suggested_next_X[j])
                run_BO_initRS_EI.append(i)
                Conductivity_BO_initRS_EI.append(suggested_next_Conductivity[j])

#             min_round = np.min(suggested_next_Conductivity)
#             min_index = np.where(suggested_next_Conductivity == min_round)
#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))
    X_BO_initRS_EI_collection.append(X_BO_initRS_EI)
    run_BO_initRS_EI_collection.append(run_BO_initRS_EI)
    Conductivity_BO_initRS_EI_collection.append(Conductivity_BO_initRS_EI)
    print('Finished e')
    
master = np.array([X_BO_initRS_EI_collection, run_BO_initRS_EI_collection, Conductivity_BO_initRS_EI_collection])
np.save('GP_BO_EI_master', master)


Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [27]:
# random sampling

X_RS_collection = []
Conductivity_RS_collection = []

for s in seed_list:

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_rs = sampler.get_samples(n_eval)
    Conductivity_RS = BO_teacher.model.predict(X_rs)[0][:,0] * input_std + input_mean
    
    

    # plot_tri(X_rs[:,0], X_rs[:,1], X_rs[:,2], Conductivity_RS, 'random sampling', True)
    X_RS_collection.append(X_rs)
    Conductivity_RS_collection.append(Conductivity_RS)
    print('Finished e')
    
master = np.array([X_RS_collection, 0, Conductivity_RS_collection])
np.save('GP_RS_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [28]:
X_BO_initRS_MPI_collection = []
run_BO_initRS_MPI_collection = []
Conductivity_BO_initRS_MPI_collection = []

for s in seed_list:
    random.seed(s)

    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_MPI = []
    run_BO_initRS_MPI = []
    Conductivity_BO_initRS_MPI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(initial_X[j])
                run_BO_initRS_MPI.append(i)
                Conductivity_BO_initRS_MPI.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'MPI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_MPI),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_MPI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_Conductivity = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            
            
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(suggested_next_X[j])
                run_BO_initRS_MPI.append(i)
                Conductivity_BO_initRS_MPI.append(suggested_next_Conductivity[j])

            min_round = np.min(suggested_next_Conductivity)
            min_index = np.where(suggested_next_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], Conductivity_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(Conductivity_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_MPI_collection.append(X_BO_initRS_MPI)
    run_BO_initRS_MPI_collection.append(run_BO_initRS_MPI)
    Conductivity_BO_initRS_MPI_collection.append(Conductivity_BO_initRS_MPI)
    print('Finished e')
    
master = np.array([X_BO_initRS_MPI_collection, run_BO_initRS_MPI_collection, Conductivity_BO_initRS_MPI_collection])
np.save('GP_BO_MPI_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [29]:
X_BO_initRS_LCB_collection = []
run_BO_initRS_LCB_collection = []
Conductivity_BO_initRS_LCB_collection = []

for s in seed_list:
    random.seed(s)
    
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_LCB = []
    run_BO_initRS_LCB = []
    Conductivity_BO_initRS_LCB = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(initial_X[j])
                run_BO_initRS_LCB.append(i)
                Conductivity_BO_initRS_LCB.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'LCB',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_LCB),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_LCB]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_Conductivity = BO_teacher.model.predict(suggested_next_X)[0][:,0] * input_std + input_mean
            
            
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(suggested_next_X[j])
                run_BO_initRS_LCB.append(i)
                Conductivity_BO_initRS_LCB.append(suggested_next_Conductivity[j])

            min_round = np.min(suggested_next_Conductivity)
            min_index = np.where(suggested_next_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], Conductivity_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(Conductivity_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_LCB_collection.append(X_BO_initRS_LCB)
    run_BO_initRS_LCB_collection.append(run_BO_initRS_LCB)
    Conductivity_BO_initRS_LCB_collection.append(Conductivity_BO_initRS_LCB)
    print('Finished e')
    
master = np.array([X_BO_initRS_LCB_collection, run_BO_initRS_LCB_collection, Conductivity_BO_initRS_LCB_collection])
np.save('GP_BO_LCB_master', master)

The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ign

In [30]:
# # grid search + random sampling
# from random import sample

# grid_linspace = np.linspace(0, 1, 50)
# grid_array = np.array([999, 999, 999])

# for i in grid_linspace:
#     for j in grid_linspace:
#         for k in grid_linspace:
#             x = np.array([i, j, k])
#             if i + j + k == 1:
#                 grid_array = np.vstack((grid_array, x))
# grid_array = grid_array[1:]

# # grid_pred_mean = BO_teacher.model.predict(grid_array)[0][:,0] * input_std + input_mean
# # grid_pred_mean_std = BO_teacher.model.predict(grid_array)[1][:,0]* input_std

# grid_pred = bst.predict(xgb.DMatrix(grid_array))

# X_grid_collection = []
# Conductivity_grid_collection = []

# for s in seed_list:
#     random.seed(s)
#     list_index = sample(list(np.arange(len(grid_pred_mean))),n_eval)
#     X_grid = []
#     Conductivity_grid = []
#     for i in list_index:
#         X_grid.append(grid_array[i])
#         Conductivity_grid.append(grid_pred_mean[i])
#     X_grid_collection.append(X_grid)
#     Conductivity_grid_collection.append(Conductivity_grid)
#     print('Finished e')
    
# master = np.array([X_grid_collection, 0, Conductivity_grid_collection])
# np.save('Grid_master', master)


In [31]:
X_BO_initRS_EI_collection = []
run_BO_initRS_EI_collection = []
Conductivity_BO_initRS_EI_collection = []

for s in seed_list:
    
    # initial_sampling = constrained random sampling = cRS
    # optimization = GpyOpt

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_EI = []
    run_BO_initRS_EI = []
    Conductivity_BO_initRS_EI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_EI.append(initial_X[j])
                run_BO_initRS_EI.append(i)
                Conductivity_BO_initRS_EI.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'EI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_EI),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_EI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch

            suggested_next_Conductivity = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_EI.append(suggested_next_X[j])
                run_BO_initRS_EI.append(i)
                Conductivity_BO_initRS_EI.append(suggested_next_Conductivity[j])

#             min_round = np.min(suggested_next_Conductivity)
#             min_index = np.where(suggested_next_Conductivity == min_round)
#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))
    X_BO_initRS_EI_collection.append(X_BO_initRS_EI)
    run_BO_initRS_EI_collection.append(run_BO_initRS_EI)
    Conductivity_BO_initRS_EI_collection.append(Conductivity_BO_initRS_EI)
    print('Finished e')
    
master = np.array([X_BO_initRS_EI_collection, run_BO_initRS_EI_collection, Conductivity_BO_initRS_EI_collection])
np.save('XGB_BO_EI_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [32]:
X_RS_collection = []
Conductivity_RS_collection = []

for s in seed_list:

    random.seed(s)
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_rs = sampler.get_samples(n_eval)

    Conductivity_RS = bst.predict(xgb.DMatrix(X_rs))
    

    # plot_tri(X_rs[:,0], X_rs[:,1], X_rs[:,2], Conductivity_RS, 'random sampling', True)
    X_RS_collection.append(X_rs)
    Conductivity_RS_collection.append(Conductivity_RS)
    print('Finished e')
    
master = np.array([X_RS_collection, 0, Conductivity_RS_collection])
np.save('XGB_RS_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [33]:
X_BO_initRS_MPI_collection = []
run_BO_initRS_MPI_collection = []
Conductivity_BO_initRS_MPI_collection = []

for s in seed_list:
    random.seed(s)

    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_MPI = []
    run_BO_initRS_MPI = []
    Conductivity_BO_initRS_MPI = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(initial_X[j])
                run_BO_initRS_MPI.append(i)
                Conductivity_BO_initRS_MPI.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'MPI',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_MPI),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_MPI]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch


            suggested_next_Conductivity = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_MPI.append(suggested_next_X[j])
                run_BO_initRS_MPI.append(i)
                Conductivity_BO_initRS_MPI.append(suggested_next_Conductivity[j])

            min_round = np.min(suggested_next_Conductivity)
            min_index = np.where(suggested_next_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], Conductivity_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(Conductivity_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_MPI_collection.append(X_BO_initRS_MPI)
    run_BO_initRS_MPI_collection.append(run_BO_initRS_MPI)
    Conductivity_BO_initRS_MPI_collection.append(Conductivity_BO_initRS_MPI)
    print('Finished e')
    
master = np.array([X_BO_initRS_MPI_collection, run_BO_initRS_MPI_collection, Conductivity_BO_initRS_MPI_collection])
np.save('XGB_BO_MPI_master', master)

Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e
Finished e


In [34]:
X_BO_initRS_LCB_collection = []
run_BO_initRS_LCB_collection = []
Conductivity_BO_initRS_LCB_collection = []

for s in seed_list:
    random.seed(s)
    
    design_space = GPyOpt.core.task.space.Design_space(space = bounds, 
                                                       constraints=constraints_baseline)
    sampler = GPyOpt.experiment_design.random_design.RandomDesign(design_space)

    X_BO_initRS_LCB = []
    run_BO_initRS_LCB = []
    Conductivity_BO_initRS_LCB = []

    for i in np.arange(rounds):
        if i == 0:
            initial_X = sampler.get_samples(batch)
            initial_Conductivity = BO_teacher.model.predict(initial_X)[0][:,0] * input_std + input_mean
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(initial_X[j])
                run_BO_initRS_LCB.append(i)
                Conductivity_BO_initRS_LCB.append(initial_Conductivity[j])

            min_round = np.min(initial_Conductivity)
            min_index = np.where(initial_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(initial_X[min_index]) + ' is '+str(min_round))
        else:
            BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                            domain = bounds,
                                            constraints = constraints_baseline,
                                            acquisition_type = 'LCB',              
                                            normalize_Y = True,
                                            X = np.array(X_BO_initRS_LCB),
                                            Y = np.array([[i] for i in Conductivity_BO_initRS_LCB]),
                                            evaluator_type = 'local_penalization',
                                            batch_size = batch,
                                            acquisition_jitter = jitter)

            suggested_next_X = BO_model.suggest_next_locations()

            assert len(suggested_next_X) == batch


            suggested_next_Conductivity = bst.predict(xgb.DMatrix(suggested_next_X))
            
            for j in np.arange(batch):
                X_BO_initRS_LCB.append(suggested_next_X[j])
                run_BO_initRS_LCB.append(i)
                Conductivity_BO_initRS_LCB.append(suggested_next_Conductivity[j])

            min_round = np.min(suggested_next_Conductivity)
            min_index = np.where(suggested_next_Conductivity == min_round)

#             print('round '+str(i)+' finished, best found at ' +str(suggested_next_X[min_index]) + ' is '+str(min_round))

    # plot_tri(np.array(X_BO_initRS_EI)[:,0], np.array(X_BO_initRS_EI)[:,1], np.array(X_BO_initRS_EI)[:,2], Conductivity_BO_initRS_EI, 'BO_EI', True)    
    # plot_BOprocess(Conductivity_BO_initRS_EI, run_BO_initRS_EI, 'BO_EI')
    X_BO_initRS_LCB_collection.append(X_BO_initRS_LCB)
    run_BO_initRS_LCB_collection.append(run_BO_initRS_LCB)
    Conductivity_BO_initRS_LCB_collection.append(Conductivity_BO_initRS_LCB)
    print('Finished e')
    
master = np.array([X_BO_initRS_LCB_collection, run_BO_initRS_LCB_collection, Conductivity_BO_initRS_LCB_collection])
np.save('XGB_BO_LCB_master', master)

The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ign